<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
from omegaconf import OmegaConf
import torch
from pathlib import Path
import argparse, gc, json, os, random, sys, time, glob, requests

%load_ext autoreload
%autoreload 2

from stable_diffusion_utils import load_model_from_config, config, generate

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
opt = config()
model = load_model_from_config(OmegaConf.load(f"{opt.config}"), f"{opt.ckpt}").to(device)
batch_idx = 0
sample_idx = 0

In [ ]:
#@title Image creation

#@markdown `batch_name`: name for subfolder and filenames<br>
#@markdown `width_height`: image dimensions<br>
#@markdown `guidance_scale`: strength of text prompt<br>
#@markdown `steps`: number of diffusion steps<br>
#@markdown `num_batch_images`: how many images you want to generate in this batch<br>
#@markdown `sampler`: KLMS is recommended<br>
#@markdown `ddim_eta`: scale of variance from 0.0 to 1.0<br>
#@markdown `seed`: use -1 for random seed or specify number manually<br>
#@markdown `init_image_or_folder`: url or path to an image, or path to a folder to pick random images from<br>
#@markdown `init_strength`: from 0.0 to 1.0 how much the init image is used<br>

#@markdown 

#@markdown Batch settings
from pathlib import Path
outputs_path = Path.home() / 'Documents/generated_data/stable_diffusion'
batch_name = "" #@param {type:"string"}
width_height = [512, 512] #@param{type: 'raw'}
guidance_scale = 7.5 #@param {type:"number"}
steps = 100 #@param {type:"integer"}
samples_per_batch = 1 # not exposed, you can do 2 or more based on GPU ram, if get CUDA out of memory need to restart runtime
num_batch_images = 5 #@param {type:"integer"}
sampler = 'klms' #@param ["klms","plms", "ddim"]
ddim_eta = 0.75 #@param {type:"number"}
seed = -1 #@param {type:"integer"}

#@markdown 

#@markdown Init image


init_image_or_folder = "" #@param {type:"string"}
init_strength = 0. #@param {type:"number"}

#@markdown 

#@markdown Prompt
prompt = "" #@param {type:"string"}


opt.init_img = init_image_or_folder
opt.ddim_steps = steps
opt.n_iter = 1
opt.n_samples = samples_per_batch
opt.outdir = outputs_path / batch_name
opt.prompt = prompt
opt.sampler = sampler
opt.scale = guidance_scale
opt.seed = random.randint(0, 2**32) if seed == -1 else seed
opt.strength = max(0.0, min(1.0, 1.0 - init_strength))
opt.W, opt.H = map(lambda x: x - x % 64, (width_height[0], width_height[1])) # resize to integer multiple of 64

if opt.strength >= 1 or init_image_or_folder == None:
    opt.init_img = ""

if opt.init_img != None and opt.init_img != '':
    opt.sampler = 'ddim'

if opt.sampler != 'ddim':
    opt.ddim_eta = 0.0

# save settings
settings = {
    'ddim_eta': ddim_eta,
    'guidance_scale': guidance_scale,
    'init_image': init_image_or_folder,
    'init_strength': init_strength,
    'num_batch_images': num_batch_images,
    'prompt': prompt,
    'sampler': sampler,
    'samples_per_batch': samples_per_batch,
    'seed': opt.seed,
    'steps': steps,
    'width': opt.W,
    'height': opt.H,
}
os.makedirs(opt.outdir, exist_ok=True)
while os.path.isfile(f"{opt.outdir}/{batch_name}({batch_idx})_settings.txt"):
    batch_idx += 1
with open(f"{opt.outdir}/{batch_name}({batch_idx})_settings.txt", "w+", encoding="utf-8") as f:
    json.dump(settings, f, ensure_ascii=False, indent=4)
sample_idx = 0

for i in range(num_batch_images):
    gc.collect()
    torch.cuda.empty_cache()

    images = generate(opt, model, batch_name, batch_idx, sample_idx)

    #clear_output(wait=True)
    print(f"Used seed: {opt.seed}")
    print(f"Saved to: {opt.outdir}")
#     for image in images:
#         display(image)

    opt.seed += 1
    sample_idx += 1
